# Evaluación de modelos Sentence Transformers (sentence similarity) para generar Text Embreddings para busqueda y recuperación asimetrica

En este cuaderno se evaluan diferentes modelos de procesamiento de lenguaje natural de similitud de sentencias para validar el entrenamiento de Tulio y Beto en el caso de uso de recuperar normas de la ley chilena comparando su titulo con preguntas.

Para ellos:
* con cada modelo se genera una base de datos vectorial con los text embeddings
* dado un set de datos de preguntas y respuestas que contiene las ids de las normas relevantes se hacen busquedas en la base de datos
* para cada pregunta se recupera de forma arbitraria las k normas (titulos de las normas representados como vectores) que tienen menor distancia coseno con la representación vectorial de la pregunta. k es el numero de normas relevantes en el set de datos para la pregunta.
* se calcula la precision de cada modelo como el promedio de la precision de cada recuperacion, esta es el porcentaje de ids de las normas recuperadas que se encuentran en el conjunto de ids relevantes del set de datos.

In [1]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import JSONLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import pandas as pd
import numpy as np
import torch

### Funcion para calcular precision de una recuperacion

In [2]:
def calculate_precision(retrieved_ids, relevant_ids):
    true_positives = retrieved_ids.intersection(relevant_ids)
    if len(retrieved_ids) == 0:
        return 0
    return len(true_positives) / len(retrieved_ids)

### Configuraciónd el set de datos de evaluación, modelos a evaluar y metricas finales.

In [3]:
df = pd.read_csv('./qa_dataset_legal_filter_heuristic_curated.csv')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
devide = 'mps' if torch.backends.mps.is_available() else device

model_names = [ 
                'sentence-transformers/all-MiniLM-L6-v2', #Modelo por defecto de ChromaDB
                #Sentence Transformers en español desde HuggingFace
                'Maite89/Roberta_finetuning_semantic_similarity_stsb_multi_mt',
                'eduardofv/stsb-m-mt-es-distilbert-base-uncased',
                'eduardofv/stsb-m-mt-es-distiluse-base-multilingual-cased-v1',
                'hiiamsid/sentence_similarity_spanish_es',
                'mrm8488/distiluse-base-multilingual-cased-v2-finetuned-stsb_multi_mt-es',
                'hackathon-pln-es/bertin-roberta-base-finetuning-esnli',
                'hackathon-pln-es/paraphrase-spanish-distilroberta',
                'jaimevera1107/all-MiniLM-L6-v2-similarity-es',
                'cnrhs/sen-sim-es',
                'dariolopez/roberta-base-bne-finetuned-msmarco-qa-es',
                'dariolopez/roberta-base-bne-finetuned-msmarco-qa-es-mnrl-mn',
                #Modelos de este trabajo
                #'emersoftware/test',
                'emersoftware/tulio-st-msmarco-es-mnrl',
                'emersoftware/beto-st-msmarco-es-mnrl',
                'emersoftware/tulio-st-msmarco-es-mnrl-v2',
                'emersoftware/beto-st-msmarco-es-mnrl-v2',
                #'emersoftware/tulio-chilean-spanish-bert-msmarco-qa-es-mnrl-mn',
                #'emersoftware/tulio-chilean-spanish-bert-finetuned-msmarco-qa-es',
                #Modelos multilingües Sentence Transformers
                'sentence-transformers/paraphrase-multilingual-mpnet-base-v2',
                'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
                'sentence-transformers/LaBSE',
                'sentence-transformers/distiluse-base-multilingual-cased-v2',
                'sentence-transformers/distiluse-base-multilingual-cased-v1'
]

metrics_results = {model_name: {'precision': []} for model_name in model_names}

### Carga de los documentos para generar las bases de datos vectoriales
Cada uno de los documentos es el titulo de una norma que tiene como metadatos las ids de las normas relevantes

In [4]:
def metadata_func (record: dict, metadata: dict) -> dict:
    metadata['idNorma'] = record.get('idNorma')
    return metadata

loader = DirectoryLoader('data_normas_json/',
                         show_progress=True,
                         use_multithreading=True,
                         max_concurrency=8,
                         loader_cls=JSONLoader,
                         loader_kwargs={
                             'jq_schema': '.',
                             'content_key': 'titulo',
                             'metadata_func': metadata_func
                            })
docs = loader.load()

100%|██████████| 784/784 [00:03<00:00, 204.88it/s]


### Evaluación de cada uno de los modelos

In [5]:
for model_name in model_names:
    print(f"Evaluando modelo: {model_name}")
    
    # Cargar y generar embeddings con el modelo actual
    embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs={'device': device})
    db = Chroma.from_documents(docs, embeddings, persist_directory=f'vectordb/{model_name}', collection_name='titulos', collection_metadata={"hnsw:space": "cosine"})
    
    for index, row in df.iterrows():
        # Comprobar si 'idNormas' es una cadena válida
        if pd.notna(row['idNormas']):
            query = row['pregunta']
            query = query.lower()
            query = query.replace('?', '').replace('¿', '').replace('!', '')
            

            relevant_ids = set(row['idNormas'].split(';'))

            retrieved_docs = db.similarity_search(query, k=len(relevant_ids))
            
            retrieved_ids = set(doc.metadata['idNorma'] for doc in retrieved_docs)

            precision = calculate_precision(retrieved_ids, relevant_ids)
            
            # Guardar métricas
            metrics_results[model_name]['precision'].append(precision)

        else:
            # Saltar esta fila si 'idNormas' no es una cadena válida
            print(f"Fila {index} omitida: idNormas es NaN o no válido.")
            continue  # Continuar con la siguiente iteración del bucles

Evaluando modelo: Maite89/Roberta_finetuning_semantic_similarity_stsb_multi_mt


/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Evaluando modelo: eduardofv/stsb-m-mt-es-distilbert-base-uncased
Evaluando modelo: eduardofv/stsb-m-mt-es-distiluse-base-multilingual-cased-v1
Evaluando modelo: hiiamsid/sentence_similarity_spanish_es
Evaluando modelo: mrm8488/distiluse-base-multilingual-cased-v2-finetuned-stsb_multi_mt-es
Evaluando modelo: hackathon-pln-es/bertin-roberta-base-finetuning-esnli
Evaluando modelo: hackathon-pln-es/paraphrase-spanish-distilroberta
Evaluando modelo: jaimevera1107/all-MiniLM-L6-v2-similarity-es
Evaluando modelo: cnrhs/sen-sim-es
Evaluando modelo: dariolopez/roberta-base-bne-finetuned-msmarco-qa-es
Evaluando modelo: dariolopez/roberta-base-bne-finetuned-msmarco-qa-es-mnrl-mn
Evaluando modelo: emersoftware/tulio-st-msmarco-es-mnrl
Evaluando modelo: emersoftware/beto-st-msmarco-es-mnrl
Evaluando modelo: emersoftware/tulio-st-msmarco-es-mnrl-v2
Evaluando modelo: emersoftware/beto-st-msmarco-es-mnrl-v2
Evaluando modelo: sentence-transformers/paraphrase-multilingual-mpnet-base-v2
Evaluando modelo:

### Se muestran los resultados de la evaluación
Precision promedio de los modelos

In [13]:
for model_name, metrics in metrics_results.items():
    avg_precision = np.mean(metrics['precision'])
    print(f"Modelo: {model_name} \t\t Precision: {avg_precision} \t ")

Modelo: Maite89/Roberta_finetuning_semantic_similarity_stsb_multi_mt 		 Precision: 0.2651335038127491 	 
Modelo: eduardofv/stsb-m-mt-es-distilbert-base-uncased 		 Precision: 0.18499406777708666 	 
Modelo: eduardofv/stsb-m-mt-es-distiluse-base-multilingual-cased-v1 		 Precision: 0.3063218490105282 	 
Modelo: hiiamsid/sentence_similarity_spanish_es 		 Precision: 0.35417300205036056 	 
Modelo: mrm8488/distiluse-base-multilingual-cased-v2-finetuned-stsb_multi_mt-es 		 Precision: 0.23016564148639623 	 
Modelo: hackathon-pln-es/bertin-roberta-base-finetuning-esnli 		 Precision: 0.2926814799456309 	 
Modelo: hackathon-pln-es/paraphrase-spanish-distilroberta 		 Precision: 0.34594218674407357 	 
Modelo: jaimevera1107/all-MiniLM-L6-v2-similarity-es 		 Precision: 0.24186450342110716 	 
Modelo: cnrhs/sen-sim-es 		 Precision: 0.35375371483862045 	 
Modelo: dariolopez/roberta-base-bne-finetuned-msmarco-qa-es 		 Precision: 0.2284735180961596 	 
Modelo: dariolopez/roberta-base-bne-finetuned-msmarco-qa

In [7]:
import numpy as np

# Calcula la precisión promedio para cada modelo
average_precision = {model: np.mean(metrics['precision']) for model, metrics in metrics_results.items()}

# Encuentra el modelo con la mejor precisión promedio
best_model = max(average_precision, key=average_precision.get)

# Imprime el nombre del modelo con mejor precisión y todos sus valores de precisión
print(f"Modelo con la mejor precisión promedio: {best_model}")
print("Valores de precisión para este modelo:")
for precision in metrics_results[best_model]['precision']:
    print(precision)

Modelo con la mejor precisión promedio: emersoftware/beto-st-msmarco-es-mnrl
Valores de precisión para este modelo:
0.25
0.2
0.5
0.5
0.6666666666666666
0.5
0.6666666666666666
0.25
0.3333333333333333
0.5
0.0
0.0
0.6
0.0
0.5
0.5
0.0
0.6
0.2
0.5
0.8
0.0
0.5
0.0
0.5
1.0
1.0
0.0
0.5
0.75
0.25
0.5
0.0
1.0
0.5
0.6666666666666666
0.2
0.6
0.6
1.0
0.5
0.5
0.3333333333333333
0.0
0.6666666666666666
0.2
0.25
0.2
0.25
0.3333333333333333
0.4
0.0
0.4
0.3333333333333333
0.0
0.0
0.25
0.8
1.0
0.5
1.0
1.0
0.5
0.0
0.0
0.6
0.2
0.75
0.5
0.0
0.5
0.3333333333333333
0.0
0.0
1.0
0.3333333333333333
0.6666666666666666
0.0
0.25
0.0
0.4
0.0
0.5
0.0
0.0
0.25
0.75
0.8571428571428571
0.3333333333333333
0.75
0.5
0.5
0.2
0.2
0.0
0.0
0.2
0.3333333333333333
0.25
0.0
0.0
0.5
1.0
1.0
0.0
0.3333333333333333
0.3333333333333333
0.6666666666666666
0.5
0.25
0.2857142857142857
0.5
0.5
0.5
0.6666666666666666
1.0
1.0
0.5
0.5
0.6666666666666666
0.3333333333333333
0.0
1.0
0.0
0.0
0.6
0.25
0.0
1.0
0.6
0.3333333333333333
0.25
0.66666666